In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [126]:
import progressbar
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from utils import *

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 2
NUM_EPOCHS = 5
NEGATIVE_SAMPLING = False

filename = "medium_text.txt"
print("Parsing text and loading training data...")
processed_text, vocab, word_to_ix, ix_to_word, training_data = load_data(filename,
                                                             CONTEXT_SIZE, model_type="cbow", subsampling=True, sampling_rate=0.001)


Parsing text and loading training data...


In [121]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.kernel = nn.Parameter(torch.rand(embedding_dim, embedding_dim))
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs, focus):
        embeds = self.embeddings(inputs)
        alpha = torch.mm(torch.mm(embeds, self.kernel), self.embeddings(focus).t())
        alpha = F.softmax(alpha)
        embeds = torch.sum(torch.mul(alpha, embeds), 0, keepdim=True)
        out = self.linear(embeds)
        log_probs = F.log_softmax(out)
        return log_probs

In [124]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [127]:
print("Starting training")
for epoch in range(NUM_EPOCHS):
    total_loss = torch.Tensor([0])
    print("Beginning epoch %d" % epoch)
    progress_bar = progressbar.ProgressBar()
    for context, target in progress_bar(training_data):
        context_var = autograd.Variable(torch.LongTensor(context))
        focus_var = autograd.Variable(torch.LongTensor([target]))
        model.zero_grad()
        log_probs = model(context_var, focus_var)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([target])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    print("Epoch %d Loss: %.5f" % (epoch, total_loss[0]))
    losses.append(total_loss)

Starting training
Beginning epoch 0
Epoch 0 Loss: 22450.66211
Beginning epoch 1
Epoch 1 Loss: 22010.00000
Beginning epoch 2
Epoch 2 Loss: 21615.87500
Beginning epoch 3
Epoch 3 Loss: 21269.98828
Beginning epoch 4
Epoch 4 Loss: 20959.13281
